In [11]:
import copy
from datetime import datetime
import json
import math
import os
from pathlib import Path
import random

from dotenv import load_dotenv
import pandas as pd
import requests

In [15]:
load_dotenv()

API_TOKEN = os.getenv('X-RapidAPI-Key')

In [19]:
def download_elo_data() -> None:
    today = datetime.today().strftime('%Y-%m-%d')
    df = pd.read_csv(f"http://api.clubelo.com/{today}")
    Path("data/elo").mkdir(parents=True, exist_ok=True)
    df = df[['Rank', 'Club', 'Country', 'Level', 'Elo']]
    today = today.replace('-', '')
    df.to_csv(f"data/elo/{today}.csv", index=False)

In [16]:
def api_get_leagues() -> None:
    url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

    params = {"current": "true"}

    headers = {
        "X-RapidAPI-Key": API_TOKEN,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)
    print(response.status_code)

    if response.json()['paging']['total'] != 1:
        raise Exception("Error: multiple pages of leagues")

    Path("data/api").mkdir(parents=True, exist_ok=True)
    with open("data/api/leagues.json", "w") as f:
        json.dump(response.json(), f)

In [17]:
def api_get_fixtures_for_league() -> None:
    pass

In [ ]:
def api_get_standings_for_league() -> None:
    pass

In [24]:
def find_league_id(country_code: str, league_name: str) -> str:
    with open("data/api/leagues.json", "r") as f:
        leagues = json.load(f)
        for league in leagues['response']:
            if league['country']['code'] == country_code and league['league']['type'] == 'League' and league['league']['name'] == league_name:
                return league['league']['id']

assert find_league_id("PL", "Ekstraklasa") == 106

In [25]:
def simulate_results() -> None:
    pass

In [ ]:
# 1 calculate or get elo
# 2 get schedule
# 3 formulate draw rules
# 4 simulate
# 5 scrape outrights odds

# european leagues with seasons not ending around May: Finland, Ireland, Norway, Sweden
# american leagues -||-: Brazil, Canada, Ecuador, USA, Uruguay
# asian leagues -||-: Japan, Korea et al.

In [9]:
table = {
    'jagiellonia': 60,
    'slask': 60,
    # 'legia': 56,
    # 'gornik': 53,
    # 'lech': 53,
    # 'rakow': 52,
    # 'pogon': 52
}

odds_df = [
    ['jagiellonia', 'warta', 1.38, 5, 7.2],
    # ['lech', 'korona', 1.82, 4.1, 3.75],
    # ['legia', 'zaglebie', 1.49, 4.4, 6.2],
    # ['pogon', 'gornik', 1.77, 4.1, 3.95],
    # ['puszcza', 'piast', 3.2, 3.3, 2.25],
    # ['radomiak', 'widzew', 2.33, 3.35, 3],
    ['rakow', 'slask', 2.38, 3.65, 2.7],
    # ['ruch', 'cracovia', 2.7, 3.5, 2.46],
    # ['lks', 'mielec', 2.75, 3.2, 2.6]
]

In [10]:
# odds_df = pd.DataFrame(columns=['team_home', 'team_away', 'OddsH', 'OddsD', 'OddsA'], data=odds_df)

# odds_df['BookieProbH'] = odds_df.apply(lambda row: 1/row['OddsH'] / (1/row['OddsH'] + 1/row['OddsD'] + 1/row['OddsA']), axis=1)
# odds_df['BookieProbD'] = odds_df.apply(lambda row: 1/row['OddsD'] / (1/row['OddsH'] + 1/row['OddsD'] + 1/row['OddsA']), axis=1)
# odds_df['BookieProbA'] = odds_df.apply(lambda row: 1/row['OddsA'] / (1/row['OddsH'] + 1/row['OddsD'] + 1/row['OddsA']), axis=1)

# odds_df['BookieProbTotal'] = odds_df.apply(lambda row: row['BookieProbH'] + row['BookieProbD'] + row['BookieProbA'], axis=1)

# odds_df.head()

,team_home,team_away,OddsH,OddsD,OddsA,BookieProbH,BookieProbD,BookieProbA,BookieProbTotal
0,jagiellonia,warta,1.38,5.00,7.2,0.681354,0.188054,0.130593,1.0
1,rakow,slask,2.38,3.65,2.7,0.394705,0.257369,0.347925,1.0


In [11]:
# start = datetime.now()

# slask_wins = 0
# sims = 100000

# for i in range(sims):
#     tmp_table = copy.deepcopy(table)
#     for index, row in odds_df.iterrows():
#         random_number = random.random()
#         if random_number < row['BookieProbH']:
#             try:
#                 tmp_table[row['team_home']] += 3
#             except KeyError:
#                 pass
#         elif random_number < row['BookieProbH'] + row['BookieProbD']:
#             try:
#                 tmp_table[row['team_home']] += 1
#             except KeyError:
#                 pass
#             try:
#                 tmp_table[row['team_away']] += 1
#             except KeyError:
#                 pass
#         else:
#             try:
#                 tmp_table[row['team_away']] += 3
#             except KeyError:
#                 pass
#     slask_won = True
#     for k, v in tmp_table.items():
#         if k != 'slask' and tmp_table['slask'] <= v:
#             slask_won = False
#     if slask_won:
#         slask_wins += 1

# print(f'Time: {datetime.now()-start}')
# print(f'{slask_wins/sims}')
# print(f'Fair odds: {math.ceil(1/(slask_wins/sims) * 100) / 100}')

Time: 0:00:14.440981
0.14505
Fair odds: 6.9
